##works cited

https://www.debuggex.com/cheatsheet/regex/python

https://docs.python.org/2/library/re.html#simulating-scanf

https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/

https://discussions.udacity.com/t/how-to-set-up-mongodb-locally-windows/185014/2

http://stackoverflow.com/questions/18836064/pymongo-method-of-getting-statistics-for-collection-byte-usage

http://stackoverflow.com/questions/3366397/delete-everything-in-a-mongodb-database

https://codefortulsa.org/2014/01/19/tulsas-first-openstreetmap-editathon/

http://stackoverflow.com/questions/12118695/efficient-way-to-remove-keys-with-empty-values-from-a-dict

http://stackoverflow.com/questions/11782566/mongodb-select-countdistinct-x-on-an-indexed-column-count-unique-results-for <--what I found when i googled 'count distinct pymongo'

https://docs.mongodb.com/getting-started/shell/query/

In [1]:
import xml.etree.cElementTree as ET
import codecs
from collections import defaultdict
import json
import re
import pprint

OSMFILE = "ex_HsHtLxQwZ2e4MjSRVh67BkZYBvVRQ.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

street_expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

osm_file = open(OSMFILE, "r")

In [2]:
#first, let's see what tags are present in the data--in case there are any surprises
import xml.etree.cElementTree as ET
import pprint
from collections import Counter

def count_tags(filename):
        alltags = []
        cnt = Counter()
        
        for event, elem in ET.iterparse(filename):
            alltags.append(elem.tag)
            
        for tag in alltags:
            cnt[tag] += 1
                
        return cnt

print "top-level tags:"
osm_file = open(OSMFILE, "r")
print count_tags(osm_file)

#from the osm documentation, tags and ways have second-level information
def count_second_level_tags(filename, secondlevel):
        alltags = []
        cnt = Counter()
        
        for event, elem in ET.iterparse(filename):
            if (elem.tag == secondlevel ):
                alltags.append(elem.attrib['k'].split(":")[0])
            
        for tag in alltags:
            cnt[tag] += 1
                
        return cnt

print "tags"
osm_file = open(OSMFILE, "r")
print count_second_level_tags(osm_file,"tag")
osm_file.close()
#finding:  'tiger' is a very common and surprising specialty tag.  'FIXME' is something I'd note on a personal to-do list.

top-level tags:
Counter({'nd': 422743, 'tag': 421160, 'node': 339454, 'way': 57047, 'member': 13658, 'relation': 925, 'bounds': 1, 'osm': 1})
tags
Counter({'tiger': 180014, 'highway': 50071, 'maxspeed': 36564, 'name': 31269, 'addr': 11769, 'lanes': 10689, 'oneway': 10185, 'building': 7038, 'access': 6780, 'service': 5578, 'name_1': 4643, 'gnis': 4036, 'source': 3677, 'amenity': 3221, 'ref': 3050, 'bicycle': 2783, 'placement': 2717, 'power': 2677, 'turn': 2412, 'foot': 2181, 'surface': 1538, 'is_in': 1512, 'landuse': 1349, 'destination': 1234, 'minspeed': 1227, 'bridge': 1165, 'layer': 1112, 'railway': 1005, 'ele': 996, 'leisure': 959, 'crossing': 953, 'type': 930, 'barrier': 923, 'hgv': 897, 'footway': 893, 'NHS': 829, 'operator': 764, 'toll': 756, 'natural': 651, 'religion': 605, 'name_2': 598, 'sport': 595, 'restriction': 587, 'parking': 571, 'waterway': 569, 'shop': 553, 'traffic_calming': 501, 'horse': 419, 'old_name': 413, 'payment': 410, 'place': 391, 'motor_vehicle': 380, 'smoot

In [3]:
#let's see if there are any tags with problematic characters, and how many third-level tags exist (tag[tag:tag])

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.match(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.match(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.search(element.attrib['k']) is not None:
            keys['problemchars'] += 1
        else:
            keys['other'] += 1

        
    return keys


def process_map_keys(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map_keys(OSMFILE)
pprint.pprint(keys)
#finding: we have zero problem characters in the data set.

{'lower': 197283, 'lower_colon': 196217, 'other': 27660, 'problemchars': 0}


In [4]:
#TODO: Consider changing code to count # of times user has contributed--code from count_tags would work
#Let's see who's contributing to the Tulsa map
def get_user(element):
    pp = pprint.PrettyPrinter(indent=4)
    user = element.get('user')
    return user


def process_map_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for child in element:
            user = get_user(child)
            if user is not None:
                users.add(user)

    return users


users = process_map_users(OSMFILE)
pprint.pprint(users)

set(['25or6to4',
     'ALeDonne',
     'AM909',
     'Aaron Lidman',
     'Adam Collier',
     'Aerdan',
     'Andrew Fish',
     'AndrewBuck',
     'AndrewSnow',
     'BCNorwich',
     'Baumi',
     'Beau Blackford',
     'Bryce C Nesbitt',
     'Carl Simonson',
     'Carlos Moreno',
     'Cato_d_Ae',
     'CenterElite',
     'Cerritus',
     'CharleyD',
     'Chetan_Gowda',
     'Chris Filip',
     'Chris Lawrence',
     'ChrisZontine',
     'ChrissW-R1',
     'CloCkWeRX',
     'Computer Repair of Tulsa',
     'Constable',
     'DK869',
     'DKNOTT',
     'DLichti',
     'Dan White',
     'DanHomerick',
     'DanPendergast',
     'Daniel Jeffries',
     'DaveHansenTiger',
     'DennisL',
     'DietCoke99',
     'ELadner',
     'Easky30',
     'ElWebDev',
     'Eljot8472',
     'Eric Godwin',
     'ErikSASSI',
     'EuleKC',
     'FIM',
     'FTA',
     'Fa7C0N',
     'FvGordon',
     'Geogast',
     'GerdP',
     'GoWestTravel',
     'GreggCarlson',
     'Groovecoder',
     'Guts Ad

In [5]:
#Personally, I abbreviate street names like Court to Ct, so I expect some of that in the Tulsa data set.
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

audit(OSMFILE)

defaultdict(set,
            {'1045': {'1045'},
             '126': {'South Harvard Avenue #126'},
             '129': {'129'},
             '167': {'Oklahoma State Highway 167'},
             '2502': {'2502'},
             '503': {'503'},
             '66': {'North Old Highway 66'},
             '97': {'State Highway 97'},
             'A': {'E 71 St #A', 'South Peoria Place, Apt A'},
             'Archer': {'Archer'},
             'Ave': {'N Lewis Ave',
              'S Harvard Ave',
              'S Peoria Ave',
              'South A Ave'},
             'Ave.': {'N. Trenton Ave.'},
             'Bouleavard': {'North Martin Luther King, Junior Bouleavard'},
             'Breckenridge': {'E Breckenridge'},
             'Brook': {'Pebble Brook'},
             'Center': {'Civic Center', 'Williams Center'},
             'Cherokee': {'S Cherokee', 'South Cherokee'},
             'Circle': {'Boise Circle',
              'Fox Run Circle',
              'North Hemlock Circle',
             

In [6]:
#there's some ugly data above, it warrants a cleanup.  Creating a little function to tidy street names.

mapping = { "Ave.": "Avenue",
            "Ave": "Avenue",
            "Bouleavard" : "Boulevard",
            "Dr." : "Drive",
            "Dr" : "Drive",
            "Rd.": "Road",
            "Rd" : "Road",
            "St": "Street",
            "St.": "Street",

            }

def update_street_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected and street_type in mapping:
            return name.replace(street_type,mapping[street_type])
        else:
            return name

In [33]:
#Oklahoma was pretty slow on using NNN-NNN-NNNN phone numbers.  
#My hometown didn't use NNN-NNNN until we were forced to in the late 90's.

numeric = re.compile(r'^([0-9])*$')
numericdashes = re.compile(r'^([0-9]|\-)*$')

def phone_number_format(element, keys):
    other_formatted_numbers = []
    problem_formatted_numbers = []
    
    if element.tag == "tag" and element.attrib['k'] == "phone":
        if numeric.match(element.attrib['v']):
            keys['numeric'] += 1
        if numericdashes.match(element.attrib['v']):
            keys['numericdashes'] += 1
        elif element.attrib['v'][0] == '+' and numericdashes.match(element.attrib['v'][1:]):
            keys['beginwplus'] += 1
        elif problemchars.search(element.attrib['v']) is not None:
            keys['problemchars'] += 1
            problem_formatted_numbers.append(element.attrib['v'])
        else:
            keys['other'] += 1
            other_formatted_numbers.append(element.attrib['v'])
    
#     if len(other_formatted_numbers) > 0:
#         print other_formatted_numbers
    
#     if len(problem_formatted_numbers) > 0:
#         print problem_formatted_numbers
    
    return keys


def process_map_phonekeys(filename):
    keys = {"numeric": 0, "numericdashes": 0, "beginwplus":0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = phone_number_format(element, keys)

    return keys

phonekeys = process_map_phonekeys(OSMFILE)
pprint.pprint(phonekeys)

{'beginwplus': 63,
 'numeric': 2,
 'numericdashes': 66,
 'other': 7,
 'problemchars': 31}


In [8]:
#there's some variation in the phone number formatting.  Let's use a phone number library to tidy these.
import phonenumbers as pn

def tidy_phone(phonenumber):
    return pn.format_number(pn.parse(phonenumber, 'US'), pn.PhoneNumberFormat.NATIONAL)

In [9]:
#first, declare the data layout we'd like:


CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

FIELDS = {
            "id": "id",
            "type": "childtype",
            "visible":"visible",
            "created": {
                      "version":"version",
                      "changeset":"changeset",
                      "timestamp":"timestamp",
                      "user":"user",
                      "uid":"uid"
                    },
            "pos": ["lat", "lon"],
            "address": {
                      "housenumber": "housenumber",
                      "postcode": "postcode",
                      "street": "street"
                    },
            "amenity": "amenity",
            "cuisine": "cuisine",
            "name": "name",
            "phone": "phone",
            "user" : "user",
            "node_refs" : "node_refs",
            "othertags" : "othertags"
            }

In [10]:
#creating a function to tidy the values by removing blank sets
def tidy_values(node):
    node = {k: v for k, v in node.items() if v!={} and v!=[]}
    return node

In [11]:
#creating a function to remove 
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        for key, value in FIELDS.iteritems():
            try:
                node[value] = element.attrib[key]
                try:
                    node['phone'] = tidy_phone(node['phone'])
                except KeyError:
                    pass
            except KeyError:
                if key == "type":
                    node['type'] = element.tag
                elif key == "created":
                    node['created'] = dict()
                    for ckey, cvalue in FIELDS[key].iteritems():
                        node['created'][cvalue] = element.attrib[ckey]
                elif key == "pos":
                    try:
                        node[key] = [float(element.attrib['lat']), float(element.attrib['lon'])]
                    except KeyError:
                        pass
                elif key == "address":
                    node['address'] = dict()
                    for akey, avalue in FIELDS[key].iteritems():
                        try:
                            node['address'][avalue] = update_street_name(element.attrib[akey])
                        except KeyError:
                            pass
                elif key == "node_refs":
                    try:
                        node['node_refs']
                    except:
                        node['node_refs'] = []
                    for nd in element.findall('nd'):
                        node['node_refs'].append(nd.attrib['ref'])
                else:
                    for tag in element.iter("tag"):
                        if tag.attrib['k'] == key:
                            node[key] = tag.attrib['v']
                        elif tag.attrib['k'][:5] == 'addr:' and tag.attrib['k'].count(":") == 1:
                            if 'address' not in node:
                                node['address'] = dict()
                            addressdetails = tag.attrib['k'].split(':')
                            node['address'][addressdetails[1]] = tag.attrib['v']
                        elif tag.attrib['k'] == 'phone':
                            node['phone'] = tidy_phone(tag.attrib['v'])
                        elif ":" not in tag.attrib['k'] and problemchars.search(tag.attrib['k']) is None:
                            node[tag.attrib['k']] = tag.attrib['v']
                        elif ':' in tag.attrib['k'] and problemchars.search(tag.attrib['k']) is None:
                            details = tag.attrib['k'].split(':')
                            try:
                                if details[0] not in node:
                                    node[details[0]] = dict()
                                node[details[0]][details[1]] = tag.attrib['v']
                            except TypeError:
                                node[details[0]] = {details[1] : tag.attrib['v']}
        node = tidy_values(node)
        return node
    else:
        return None

In [12]:
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [13]:
# NOTE: if you are running this code on your computer, with a larger dataset, 
# call the process_map procedure with pretty=False. The pretty=True option adds 
# additional spaces to the output, making it significantly larger.
data = process_map(OSMFILE, True)

In [18]:
pprint.pprint(data[0])

{'barrier': 'gate',
 'created': {'changeset': '7252326',
             'timestamp': '2011-02-11T06:46:06Z',
             'uid': '92286',
             'user': 'Paul Johnson',
             'version': '2'},
 'id': '102756508',
 'pos': [36.021577, -95.9037904],
 'type': 'node',
 'user': 'Paul Johnson'}


In [19]:
##New file made, time to pull it into MongoDB
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

db = get_db('TulsaOSM')

##run in command line:
##C:\Program Files\MongoDB\Server\3.4\bin>mongoimport /d TulsaOSM /c OSM /file:"C:\Users\agdje\OneDrive\Documents\Udacity\Nanodegree\DataAnalysis\P3WrangleOpenStreetMap\ex_HsHtLxQwZ2e4MjSRVh67BkZYBvVRQ.osm.json"

In [34]:
#now that it's imported, what's the size
print db.command('dbstats')

{u'storageSize': 12107776.0, u'ok': 1.0, u'avgObjSize': 278.54168841944914, u'views': 0, u'db': u'TulsaOSM', u'indexes': 1, u'objects': 396501, u'collections': 1, u'numExtents': 0, u'dataSize': 110442058.0, u'indexSize': 1396736.0}


In [39]:
def user_pipeline():
    pipeline = [
        { "$match" : { "user" : { "$exists" : True }}}, ##only count changes with a username attached
        { "$group" : { "_id" : "$user", "count": { "$sum" : 1 }}},
        { "$sort" : { "count" : -1 }},
        { "$limit" : 1 }
    ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.OSM.aggregate(pipeline)]

In [38]:
print db.OSM.find_one()

{u'barrier': u'gate', u'created': {u'changeset': u'7252326', u'user': u'Paul Johnson', u'version': u'2', u'uid': u'92286', u'timestamp': u'2011-02-11T06:46:06Z'}, u'pos': [36.021577, -95.9037904], u'user': u'Paul Johnson', u'_id': ObjectId('58699e8267971b4d0dac7e0d'), u'type': u'node', u'id': u'102756508'}


In [40]:
pipeline = user_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result[0])

{u'_id': u'Paul Johnson', u'count': 133819}


##FINDING 
Paul Johnson, the user with the highest number of changes, sounded familiar so I looked -- in 2014 he had over a million nodes in total changes
https://codefortulsa.org/2014/01/19/tulsas-first-openstreetmap-editathon/

In [41]:
#how many total users have contributed? COUNT DISTINCT equivalent
def count_users_pipeline():
    pipeline = [
        { "$match" : { "user" : { "$exists" : True }}}, ##only count changes with a username attached
        { "$group" : { "_id" : "$user", "count": { "$sum" : 1 }}},
        { "$group" : { "_id" : "unique users", "count" : { "$sum" : 1 }}}
    ]
    return pipeline

In [42]:
pipeline = count_users_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result[0])

{u'_id': u'unique users', u'count': 371}


In [44]:
#number of nodes
db.OSM.find( { "type" : "node" }).count()

339454

In [45]:
#number of ways
db.OSM.find( { "type" : "way" }).count()

57047

In [46]:
#There's a running local joke about whether there are more churches or bars in Tulsa.
def count_amenities_pipeline():
    pipeline = [
        { "$match" : {"amenity" : { "$exists" : True }, "type" : "node" }}, #only nodes with an amenity tag
        { "$group" : { "_id" : "$amenity", "count": {"$sum" : 1 }}},
        { "$sort" : { "count" : -1 }}
    ]
    return pipeline

In [47]:
pipeline = count_amenities_pipeline()
result = aggregate(db, pipeline)
# pprint.pprint(result)
pprint.pprint(result[0:5])

[{u'_id': u'place_of_worship', u'count': 554},
 {u'_id': u'school', u'count': 140},
 {u'_id': u'fast_food', u'count': 92},
 {u'_id': u'restaurant', u'count': 76},
 {u'_id': u'fountain', u'count': 67}]


In [53]:
#bars aren't even in the top five in this data set.  I'm selecting out places likely to be attended solely for drinking vs churches.
def count_certain_amenities_pipeline():
    pipeline = [
        { "$match" : {"amenity" : { "$in" : ["place_of_worship", "bar", "pub", "liquor", "nightclub"]} , "type" : "node" }}, #only the nodes we're interested in
        { "$group" : { "_id" : "$amenity", "count": {"$sum" : 1 }}},
        { "$sort" : { "count" : -1 }}
    ]
    return pipeline

In [54]:
pipeline = count_certain_amenities_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result)

[{u'_id': u'place_of_worship', u'count': 554},
 {u'_id': u'bar', u'count': 9},
 {u'_id': u'pub', u'count': 8},
 {u'_id': u'liquor', u'count': 2},
 {u'_id': u'nightclub', u'count': 1}]
